In [1]:
%cd ..

/home/maverick/Projects/ml-scratchpad


/home/maverick/Projects/ml-scratchpad/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [33]:
!ls src/data

__init__.py  __pycache__/


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [35]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import albumentations as A

from torch.utils.data import DataLoader, default_collate
from transformers import DataCollatorWithPadding
from src.data.core_datasets import PhraseCutDataset

In [22]:
ds = PhraseCutDataset(
    data_root="/run/media/maverick/Backup/datasets/phrasecut",
    task_json_path="refer_train.json",
    tokenizer_pretrained_path="openai/clip-vit-base-patch32",
    prompt_method="shuffle+",
    neg_prob=0.2,
    return_tensors="pt"
)

In [58]:
class CustomDataCollatorWithPadding(DataCollatorWithPadding):
    def __init__(self, padding_keys, *args, **kwargs):
        self.padding_keys = set(padding_keys)

        super().__init__(*args, **kwargs)

    def __call__(self, batch):
        first_example = batch[0]
        padding_features = [
            {key: value for key, value in example.items() if key in self.padding_keys}
            for example in batch
        ]

        padded_features = super().__call__(padding_features)

        no_padding_features = [
            {key: value for key, value in example.items() if key not in padded_features}
            for example in batch
        ]

        collated_features = default_collate(no_padding_features)
        
        return {**collated_features, **padded_features}  

In [59]:
dl = DataLoader(ds, num_workers=0, batch_size=1, collate_fn=CustomDataCollatorWithPadding(padding_keys=["input_ids", "attention_mask"], tokenizer=ds.tokenizer))

In [60]:
first = next(iter(dl))
first

['_MutableMapping__marker', '__abstractmethods__', '__class__', '__class_getitem__', '__contains__', '__copy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__ior__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__or__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__ror__', '__setattr__', '__setitem__', '__setstate__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_encodings', '_n_sequences', 'char_to_token', 'char_to_word', 'clear', 'convert_to_tensors', 'copy', 'data', 'encodings', 'fromkeys', 'get', 'is_fast', 'items', 'keys', 'n_sequences', 'pop', 'popitem', 'sequence_ids', 'setdefault', 'to', 'token_to_chars', 'token_to_sequence', 'token_to_word', 'tokens', 'update', 'values', 'word_ids', 'word_to_chars', 'word_to_toke

{'image': tensor([[[[105,  69,  43],
           [ 84,  52,  31],
           [ 64,  38,  21],
           ...,
           [120,  29,   2],
           [106,  50,  17],
           [119,  57,  34]],
 
          [[106,  70,  44],
           [ 87,  55,  32],
           [ 69,  42,  25],
           ...,
           [114,  51,  18],
           [104,  45,  13],
           [114,  28,  11]],
 
          [[105,  69,  43],
           [ 89,  57,  34],
           [ 72,  45,  26],
           ...,
           [ 75,  25,   0],
           [103,  63,  37],
           [117,  81,  57]],
 
          ...,
 
          [[ 80,  48,  25],
           [ 83,  51,  28],
           [ 81,  49,  26],
           ...,
           [107,  64,  30],
           [108,  65,  31],
           [107,  64,  30]],
 
          [[ 80,  48,  25],
           [ 82,  50,  29],
           [ 79,  47,  26],
           ...,
           [108,  65,  31],
           [109,  66,  32],
           [109,  66,  32]],
 
          [[ 81,  49,  26],
           

In [56]:
first["input_ids"]

tensor([[49406,   320,  1125,   539,  4287,  2912,  3814,   269, 49407]])